In [3]:
import pandas as pd 
import numpy as np 
import json

In [4]:
df_cards = pd.read_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Data/cards_data.csv')
df_transactions = pd.read_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Data/transactions_data.csv')
df_users = pd.read_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Data/users_data.csv')

with open('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Data/mcc_codes.json', 'r', encoding = 'utf-8') as f:
    data = json.load(f)
df_mcc = pd.DataFrame(data.items(), columns = ['mcc_code', 'name'])

### Cards data

In [5]:
df_cards.head(3)

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No


In [6]:
df_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     6146 non-null   int64 
 1   client_id              6146 non-null   int64 
 2   card_brand             6146 non-null   object
 3   card_type              6146 non-null   object
 4   card_number            6146 non-null   int64 
 5   expires                6146 non-null   object
 6   cvv                    6146 non-null   int64 
 7   has_chip               6146 non-null   object
 8   num_cards_issued       6146 non-null   int64 
 9   credit_limit           6146 non-null   object
 10  acct_open_date         6146 non-null   object
 11  year_pin_last_changed  6146 non-null   int64 
 12  card_on_dark_web       6146 non-null   object
dtypes: int64(6), object(7)
memory usage: 624.3+ KB


In [ ]:
# Check for duplicated values 
df_cards.duplicated().sum()

0

In [ ]:
# Check for missing values
df_cards.isnull().sum()

id                       0
client_id                0
card_brand               0
card_type                0
card_number              0
expires                  0
cvv                      0
has_chip                 0
num_cards_issued         0
credit_limit             0
acct_open_date           0
year_pin_last_changed    0
card_on_dark_web         0
dtype: int64

No duplicated or missing values were found in the data.

In [ ]:
# Convert 'expires' and 'acct_open_date' columns to datetime
df_cards['expires'] = pd.to_datetime(df_cards['expires'])
df_cards['acct_open_date'] = pd.to_datetime(df_cards['acct_open_date'])

# Remove '$' sign from 'credit_limit' column and convert to numeric
df_cards['credit_limit'] = df_cards['credit_limit'].str.replace('$', '')
df_cards['credit_limit'] = pd.to_numeric(df_cards['credit_limit'])

# Check data types
df_cards[['expires', 'acct_open_date', 'credit_limit']].dtypes

C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_1800\4199121220.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_cards['credit_limit'] = df_cards['credit_limit'].str.replace('$', '')


expires           datetime64[ns]
acct_open_date    datetime64[ns]
credit_limit               int64
dtype: object

In [ ]:
# Create a numeric key for the date column
df_cards['acct_open_date_key'] = df_cards['acct_open_date'].dt.strftime('%Y%m%d').astype(int)

In [ ]:
# Drop unnecessary columns
df_cards.drop(columns = ['expires', 'cvv', 'acct_open_date', 'card_on_dark_web'], inplace = True)

In [12]:
df_cards = df_cards.rename(columns={"id": "card_id"})

In [13]:
df_cards.head()

,card_id,client_id,card_brand,card_type,card_number,has_chip,num_cards_issued,credit_limit,year_pin_last_changed,acct_open_date_key
0,4524,825,Visa,Debit,4344676511950444,YES,2,24295,2008,20020901
1,2731,825,Visa,Debit,4956965974959986,YES,2,21968,2014,20140401
2,3701,825,Visa,Debit,4582313478255491,YES,2,46414,2004,20030701
3,42,825,Visa,Credit,4879494103069057,NO,1,12400,2012,20030101
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,YES,1,28,2009,20080901


In [ ]:
# Define the list of columns to keep in the final csv 
cols_card = ['card_id', 'card_brand', 'card_type', 'has_chip',
             'num_cards_issued', 'credit_limit', 'year_pin_last_changed', 'acct_open_date_key']

# Save the df to csv 
df_cards.to_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Processed/cards_clean.csv', index = False, columns = cols_card, encoding = 'utf-8')

### Transactions data

In [15]:
df_transactions.head(3)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN


In [16]:
df_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13305915 entries, 0 to 13305914
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            object 
 2   client_id       int64  
 3   card_id         int64  
 4   amount          object 
 5   use_chip        object 
 6   merchant_id     int64  
 7   merchant_city   object 
 8   merchant_state  object 
 9   zip             float64
 10  mcc             int64  
 11  errors          object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.2+ GB


In [ ]:
# Check for duplicated values
df_transactions.duplicated().sum()

0

In [ ]:
# Check for missing values 
df_transactions.isnull().sum()

id                       0
date                     0
client_id                0
card_id                  0
amount                   0
use_chip                 0
merchant_id              0
merchant_city            0
merchant_state     1563700
zip                1652706
mcc                      0
errors            13094522
dtype: int64

Missing values were found in 3 columns:
- merchant_state: 1563700 missing
- zip: 1652706 missing
- errors: 13094522 missing

In [ ]:
# Convert 'date' column to datetime
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

# Remove '$' sign from 'amount' column and convert to numeric
df_transactions['amount'] = df_transactions['amount'].str.replace('$', '')
df_transactions['amount'] = pd.to_numeric(df_transactions['amount'])

# Check data types
df_transactions[['date' ,'amount']].dtypes

C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_1800\3100986734.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_transactions['amount'] = df_transactions['amount'].str.replace('$', '')


date      datetime64[ns]
amount           float64
dtype: object

In [ ]:
# Check all the unique values in the 'errors' column
df_transactions['errors'].unique()

array([nan, 'Technical Glitch', 'Bad Expiration', 'Bad Card Number',
       'Insufficient Balance', 'Bad PIN', 'Bad CVV', 'Bad Zipcode',
       'Insufficient Balance,Technical Glitch',
       'Bad PIN,Insufficient Balance', 'Bad PIN,Technical Glitch',
       'Bad Expiration,Technical Glitch',
       'Bad Card Number,Bad Expiration',
       'Bad Card Number,Insufficient Balance',
       'Bad Expiration,Insufficient Balance', 'Bad Card Number,Bad CVV',
       'Bad CVV,Technical Glitch', 'Bad Expiration,Bad CVV',
       'Bad CVV,Insufficient Balance', 'Bad Card Number,Technical Glitch',
       'Bad Zipcode,Insufficient Balance',
       'Bad Card Number,Bad Expiration,Insufficient Balance',
       'Bad Zipcode,Technical Glitch'], dtype=object)

It is reasonable to assume that the majority of transactions are error-free, given that errors occur relatively infrequently. Therefore, missing values in the 'errors' column are treated as 'No Error'.

In [21]:
df_transactions['errors'].fillna('No Error', inplace = True)

In [ ]:
# Filling missing 'merchant_state' with mode and 'zip' with mean to keep consistency
df_transactions['merchant_state'].fillna(df_transactions['merchant_state'].mode()[0], inplace = True)
df_transactions['zip'].fillna(df_transactions['zip'].mean(), inplace = True)

In [ ]:
# Check for missing values again 
df_transactions.isnull().sum()

id                0
date              0
client_id         0
card_id           0
amount            0
use_chip          0
merchant_id       0
merchant_city     0
merchant_state    0
zip               0
mcc               0
errors            0
dtype: int64

In [ ]:
# Create a numeric key for the date column
df_transactions['date_key'] = df_transactions['date'].dt.strftime('%Y%m%d').astype(int)

In [ ]:
# Drop unnecessary columns
df_transactions.drop(columns = 'date', inplace = True)

In [ ]:
df_transactions = df_transactions.rename(columns = {'id': 'transaction_id',
                                                    'client_id': 'user_id',
                                                    'mcc': 'mcc_code'})

In [30]:
df_transactions.head()

,transaction_id,user_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc_code,errors,date_key
0,7475327,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No Error,20100101
1,7475328,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,No Error,20100101
2,7475329,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,No Error,20100101
3,7475331,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,No Error,20100101
4,7475332,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,No Error,20100101


In [ ]:
# Define the list of columns to keep in the final csv 
cols_transaction = ['transaction_id', 'user_id', 'card_id', 'merchant_id',
                    'mcc_code', 'date_key', 'amount', 'use_chip', 'errors']

# Save the df to csv 
df_transactions.to_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Processed/transactions_clean.csv', index = False, columns = cols_transaction, encoding = 'utf-8')

### Users data 

In [32]:
df_users.head(3)

,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5


In [33]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   current_age        2000 non-null   int64  
 2   retirement_age     2000 non-null   int64  
 3   birth_year         2000 non-null   int64  
 4   birth_month        2000 non-null   int64  
 5   gender             2000 non-null   object 
 6   address            2000 non-null   object 
 7   latitude           2000 non-null   float64
 8   longitude          2000 non-null   float64
 9   per_capita_income  2000 non-null   object 
 10  yearly_income      2000 non-null   object 
 11  total_debt         2000 non-null   object 
 12  credit_score       2000 non-null   int64  
 13  num_credit_cards   2000 non-null   int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 218.9+ KB


In [ ]:
# Check for duplicated values
df_users.duplicated().sum()

0

In [ ]:
# Check for missing values
df_users.isnull().sum()

id                   0
current_age          0
retirement_age       0
birth_year           0
birth_month          0
gender               0
address              0
latitude             0
longitude            0
per_capita_income    0
yearly_income        0
total_debt           0
credit_score         0
num_credit_cards     0
dtype: int64

In [ ]:
# Remove '$' sign from 'per-capita_income', 'yearly_income' and 'total_debt' columns and convert to numeric
df_users['per_capita_income'] = df_users['per_capita_income'].str.replace('$', '')
df_users['per_capita_income'] = pd.to_numeric(df_users['per_capita_income'])

df_users['yearly_income'] = df_users['yearly_income'].str.replace('$', '')
df_users['yearly_income'] = pd.to_numeric(df_users['yearly_income'])

df_users['total_debt'] = df_users['total_debt'].str.replace('$', '')
df_users['total_debt'] = pd.to_numeric(df_users['total_debt'])

C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_1800\718438569.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_users['per_capita_income'] = df_users['per_capita_income'].str.replace('$', '')
C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_1800\718438569.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_users['yearly_income'] = df_users['yearly_income'].str.replace('$', '')
C:\Users\FPTSHOP\AppData\Local\Temp\ipykernel_1800\718438569.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_users['total_debt'] = df_u

In [ ]:
# Check data types
df_users[['per_capita_income', 'yearly_income', 'total_debt']].dtypes

per_capita_income    int64
yearly_income        int64
total_debt           int64
dtype: object

In [ ]:
# Check unique values in 'gender' column to ensure consistency
df_users['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [ ]:
# Drop unnecessary columns
df_users.drop(columns = ['birth_year', 'birth_month'], inplace = True)

In [40]:
df_users = df_users.rename(columns = {'id': 'user_id'})

In [41]:
df_users.head()

,user_id,current_age,retirement_age,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,Female,462 Rose Lane,34.15,-117.76,29278,59696,127613,787,5
1,1746,53,68,Female,3606 Federal Boulevard,40.76,-73.74,37891,77254,191349,701,5
2,1718,81,67,Female,766 Third Drive,34.02,-117.89,22681,33483,196,698,5
3,708,63,63,Female,3 Madison Street,40.71,-73.99,163145,249925,202328,722,4
4,1164,43,70,Male,9620 Valley Stream Drive,37.76,-122.44,53797,109687,183855,675,1


In [ ]:
# Define the list of columns to keep in the final csv 
cols_user = ['user_id', 'gender', 'current_age', 'retirement_age', 'address',
             'latitude', 'longitude', 'per_capita_income', 'yearly_income',
             'total_debt', 'credit_score', 'num_credit_cards']

# Save the df to csv 
df_users.to_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Processed/users_clean.csv', index = False, columns = cols_user, encoding = 'utf-8')

### MCC code

In [43]:
df_mcc.head(3)

,mcc_code,name
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"


In [44]:
df_mcc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   mcc_code  109 non-null    object
 1   name      109 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [ ]:
# Check for duplicated values
df_mcc.duplicated().sum()

0

In [ ]:
# Check for missing values
df_mcc.isnull().sum()

mcc_code    0
name        0
dtype: int64

In [ ]:
# Save the df to csv
df_mcc.to_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Processed/mcc_clean.csv', index = False, encoding = 'utf-8')

### Dim Merchant

In [ ]:
# Create merchant dimension table from transactions
df_merchants = df_transactions[['merchant_id', 'merchant_city', 'merchant_state', 'zip']].copy()

In [ ]:
# Drop duplicated values based on 'merchant_id'
df_merchants = df_merchants.drop_duplicates(subset=['merchant_id'])

In [ ]:
# Check for duplicated values
df_merchants.duplicated().sum()

0

In [51]:
df_merchants.head()

,merchant_id,merchant_city,merchant_state,zip
0,59935,Beulah,ND,58523.0
1,67570,Bettendorf,IA,52722.0
2,27092,Vista,CA,92084.0
4,13051,Harwood,MD,20776.0
5,20519,Bronx,NY,10464.0


In [ ]:
# Save the df to csv
df_merchants.to_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Processed/merchants_clean.csv', index = False, encoding = 'utf-8')

### Dim Date

In [ ]:
# Create date dimension table

date_range = pd.date_range(start="2000-01-01", end="2030-12-31")

dim_date = pd.DataFrame({
    "date_key": date_range.strftime("%Y%m%d").astype(int),
    "full_date": date_range,
    "year": date_range.year,
    "quarter": date_range.quarter,
    "month": date_range.month,
    "day": date_range.day,
    "weekday": date_range.strftime("%A"),
})

In [54]:
dim_date.head()

,date_key,full_date,year,quarter,month,day,weekday
0,20000101,2000-01-01,2000,1,1,1,Saturday
1,20000102,2000-01-02,2000,1,1,2,Sunday
2,20000103,2000-01-03,2000,1,1,3,Monday
3,20000104,2000-01-04,2000,1,1,4,Tuesday
4,20000105,2000-01-05,2000,1,1,5,Wednesday


In [ ]:
# Save the df to csv 
dim_date.to_csv('C:/Users/FPTSHOP/Tài liệu/Financial Analysis/Processed/dimdate.csv', index = False, encoding = 'utf-8')